# COW System
### Objective

DESCRIBE THE COW SYSTEM HERE

### Table of Contents
<ul>
    <li><a href='#gathering'>Gathering</a></li>
    <li><a href='#analyze_cow'>Analysis, COW State System Data</a></li>
    <li><a href='#clean_cow'>Cleaning, COW State System Data</a></li>
    <li><a href='#insert_cow'>Insert, COW State System Data</a></li>
    <li><a href='#analyze_major'>Analysis, Major Powers Data</a></li>
    <li><a href='#clean_major'>Cleaning, Major Powers Data</a></li>
    <li><a href='#insert_major'>Insert, Major Powers Data</a></li>
</ul>

### To Do
- Add COW State System designations (`society_group`: 6) in `society_to_society_group`
- Add Major Power designations (`society_group`: 7) in `society_to_society_group`
- Truncate tables and rerun both notebooks

In [37]:
import pandas as pd
import datetime as dt

In [3]:
# Import classes and functions needed for this analysis from config module
# These are only available on my computer
from config import dbaccess, validator, society, error_dict_to_string

# Create an instance of the DBAccess class for running queries
db = dbaccess.DBAccess()

# Create an instance of the Validator class for validating data prior to inserting/updating database
val = validator.Validator()

<a id='gathering'></a>
## Data Gathering
### State Data
Read in the COW Country Codes and match up with existing societies in the database, which was initially created from UN data.

In [19]:
codes = pd.read_csv('cow_country_codes.csv')
codes.drop_duplicates(inplace=True)
codes.rename(columns={'StateAbb': 'abbreviation', 'CCode': 'cow_code', 'StateNme': 'country_name'}, inplace=True)
codes.sample(5)

,abbreviation,cow_code,country_name
206,PRK,731,North Korea
17,SKN,60,St. Kitts and Nevis
218,THI,800,Thailand
34,PAR,150,Paraguay
7,DOM,42,Dominican Republic


Add "Federal Republic of Central America" with `cow_code` of 89. This value is in the `indep_from` column for Guatemala, Honduras, El Salvador, Nicaragua, and Costa Rica in another COW dataset, but isn't in the `country_codes` dataset.

In [20]:
frca = {'abbreviation':'FRC', 
        'cow_code':89, 
        'country_name': 'Federal Republic of Central America'}
codes = codes.append(frca, ignore_index=True)

#### Match COW Countries with `society` Table
Many countries already exist in the `society` database table, based on United Nations data. Match COW Project countries with those in the database and execute an UPDATE query to set their `cow_code` field to the corresponding value in the `codes` dataframe. Add countries in the COW data that are not in the database to `society`.

In [22]:
query = db.run_query('SELECT society_id, common_name FROM society')
societies = pd.DataFrame.from_dict(query['data'])
societies.sample(5)

,society_id,common_name
35,36,Burundi
244,245,Western Sahara
83,84,Ghana
54,55,Curaçao
107,108,Israel


Find entries in `codes` that are not in the `societies` dataframe (i.e., not in the `society` database table).

In [23]:
print(codes[~codes['country_name'].isin(list(societies['common_name'].unique()))].shape)
codes[~codes['country_name'].isin(list(societies['common_name'].unique()))]

(41, 3)


,abbreviation,cow_code,country_name
0,USA,2,United States of America
2,BHM,31,Bahamas
11,SLU,56,St. Lucia
12,SVG,57,St. Vincent and the Grenadines
13,AAB,58,Antigua & Barbuda
14,SKN,60,St. Kitts and Nevis
37,NTH,210,Netherlands
47,HAN,240,Hanover
48,BAV,245,Bavaria
50,GFR,260,German Federal Republic


Fifteen of these 41 unmatched countries are already in `societies`, but `country_name` in `codes` is different from `common_name` in `societies`. Update `country_name` in `codes` dataframe to match `common_name`. Create two dataframes, one for countries already in the database that need to be updated with COW country codes and one for countries that need to be added to the database.

In [24]:
# Update country_name to match common_name in the database
update_cow = {
    'United States of America': 'USA',
    'Bahamas': 'The Bahamas',
    'St. Lucia': 'Saint Lucia',
    'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
    'Antigua & Barbuda': 'Antigua and Barbuda',
    'St. Kitts and Nevis': 'Saint Kitts and Nevis',
    'Netherlands': 'The Netherlands',
    'Czech Republic': 'Czechia',
    'Cape Verde': 'Cabo Verde',
    'Sao Tome and Principe': 'São Tomé and Príncipe',
    'Ivory Coast': 'Côte d’Ivoire',
    'Democratic Republic of the Congo': 'DRC',
    'Swaziland': 'Eswatini',
    'East Timor': 'Timor-Leste',
    'Federated States of Micronesia': 'Micronesia'
}
codes.replace({'country_name': update_cow}, inplace=True)

####### Create two dataframes
# Societies already in the database that need to be updated with COW country code
# Use merge to get countries with data in database and country_codes dataframe
existing_societies = codes.merge(societies, left_on='country_name', right_on='common_name', how='inner')

# Societies that need to be added to the database
# Find country_name in country_codes dataframe that aren't in existing_societies
new_societies = codes[~codes['country_name'].isin(list(existing_societies['country_name'].unique()))].copy()

In [25]:
# 26 new_societies and 192 existing_societies
print('New:', new_societies.shape[0])
print('Existing:', existing_societies.shape[0])

New: 26
Existing: 192


#### Update `society` Table
`1)` Update `society` table in database: Set `cow_code` for records in `existing_societies`.

`2)` Create necessary information for records in `new_societies` and add to `society` table.

In [26]:
# Check if any of the values fall outside of the 0-999 range for COW country codes
print('Number of errors:', existing_societies[val.integer_out_of_bounds(existing_societies['cow_code'],0,999)].shape[0])

Number of errors: 0


In [27]:
# Build an UPDATE statement for each row, then execute them
def build_update_statement(row):
    return 'UPDATE society SET cow_code = ' + str(row['cow_code']) + ' WHERE society_id = ' + str(row['society_id'])

# Create UPDATE statement and execute
existing_societies['update_statement'] = existing_societies.apply(build_update_statement, axis=1)

records_updated = 0
row = 1
print('Attempting update of', existing_societies.shape[0], 'rows')
for update in existing_societies['update_statement']:
    update_query = db.run_query(update)
    records_updated += update_query['rows']
    print(row, end=' ')
    row += 1

print('\n', records_updated, 'records updated')

#print( 'This has already been run and 192 records were updated.' )

Attempting update of 192 rows
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 
 192 records updated


In [30]:
# Pull a few records of society data from the database for a confirmation check
query = db.run_query('SELECT * FROM society')
peek = pd.DataFrame.from_dict(query['data'])
peek.sample(5)

,society_id,common_name,official_name,capital,society_type_id,un_m49,iso_alpha3,un_region,cow_code,still_exists
4,142,Micronesia,Federated States of Micronesia,Palikir,1,583,FSM,057,987,0
3,5,Andorra,Principality of Andorra,Andorra la Vella,1,020,AND,039,232,0
1,128,Macao,Macao Special Administrative Region of the Peo...,Macao,1,446,MAC,030,0,0
0,21,Belgium,Kingdom of Belgium,Brussels,1,056,BEL,155,211,0
2,130,Madagascar,Republic of Madagascar,Antananarivo,1,450,MDG,014,580,0


##### Add New Societies
Many of these societies had multiple different names and constitutional types (such as duchies, kingdoms, etc). For now, add to the database in barebones format with common name, official name (duplicated from common name), society type ID of 99 ("Uncategorized"), and COW Code. These will be updated later.

In [31]:
# Drop abbreviation column and rename country_name to common_name
new_societies.drop(columns=['abbreviation'], inplace=True)
new_societies.rename(columns={'country_name':'common_name'}, inplace=True)

# Add official_name as duplicate of common_name, and set society_type_id to 99
new_societies['official_name'] = new_societies['common_name']
new_societies['society_type_id'] = 99

new_societies.sample(5)

,cow_code,common_name,official_name,society_type_id
71,337,Tuscany,Tuscany,99
217,89,Federal Republic of Central America,Federal Republic of Central America,99
67,329,Two Sicilies,Two Sicilies,99
196,817,Republic of Vietnam,Republic of Vietnam,99
53,269,Saxony,Saxony,99


In [32]:
# Check for errors in the new_societies dataframe before insertion
print(error_dict_to_string(society.Society.validate_societies(new_societies, db, val)))

No errors


In [33]:
# Insert Societies
print('Expecting to insert:', new_societies.shape[0], 'rows') # How many rows am I expecting to be inserted?
insert_new_societies = db.run_query(db.build_insert_query('society', new_societies))
print(insert_new_societies['rows'], 'records inserted')

Expecting to insert: 26 rows
26 records inserted


### COW State System Membership Data
Read in data containing entry and exit dates in the Correlates of War Project state system.

In [72]:
states = pd.read_csv('states2016.csv')
states.sample(5)

,stateabb,ccode,statenme,styear,stmonth,stday,endyear,endmonth,endday,version
184,LEB,660,Lebanon,1946,3,10,2016,12,31,2016
79,SIC,329,Two Sicilies,1816,1,1,1861,2,13,2016
3,CUB,40,Cuba,1902,5,20,1906,9,25,2016
159,MAW,553,Malawi,1964,7,6,2016,12,31,2016
233,SOL,940,Solomon Islands,1978,7,7,2016,12,31,2016


### Major Power Data
Read in dataset to designate major powers.

In [6]:
majors = pd.read_csv('majors2016.csv')
majors.sample(5)

,stateabb,ccode,styear,stmonth,stday,endyear,endmonth,endday,version
2,FRN,220,1816,1,1,1940,6,22,2016
6,GMY,255,1991,12,11,2016,12,31,2016
10,RUS,365,1922,1,1,2016,12,31,2016
13,JPN,740,1991,12,11,2016,12,31,2016
12,JPN,740,1895,4,1,1945,8,14,2016


<a id='analyze_cow'></a>
## Analysis, COW State System Data

In [36]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   stateabb  243 non-null    object
 1   ccode     243 non-null    int64 
 2   statenme  243 non-null    object
 3   styear    243 non-null    int64 
 4   stmonth   243 non-null    int64 
 5   stday     243 non-null    int64 
 6   endyear   243 non-null    int64 
 7   endmonth  243 non-null    int64 
 8   endday    243 non-null    int64 
 9   version   243 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 19.1+ KB


#### Issues Found:
`1)` Create two columns - `start_date` and `end_date` - using `styear`, `stmonth`, `stday`, `endyear`, `endmonth`, and `endday`.

`2)` Dataframe uses `cow_code` as identifier, while the database uses an auto-generated `society_id`.  Match on `cow_code` so database inserts will use correct identifier.

`3)` Drop unneeded columns.

`4)` Add column `society_group_id` and set to 6 (ID for 'COW State System') for all rows.

<a id='clean_cow'></a>
## Clean, COW State System Data
### `1)` Create `start_date` and `end_date` columns
Start date is stored in columns `styear`, `stmonth`, and `stday`.  End date is stored in columns `endyear`, `endmonth`, and `endday`.

In [73]:
states['start_date'] = pd.to_datetime(states[['styear','stmonth','stday']].astype(str).agg('-'.join, axis=1))
states['end_date'] = pd.to_datetime(states[['endyear','endmonth','endday']].astype(str).agg('-'.join, axis=1))

#### Test

In [74]:
states.sample(5)

,stateabb,ccode,statenme,styear,stmonth,stday,endyear,endmonth,endday,version,start_date,end_date
0,USA,2,United States of America,1816,1,1,2016,12,31,2016,1816-01-01,2016-12-31
184,LEB,660,Lebanon,1946,3,10,2016,12,31,2016,1946-03-10,2016-12-31
71,AUS,305,Austria,1955,7,27,2016,12,31,2016,1955-07-27,2016-12-31
34,PAR,150,Paraguay,1846,1,1,1870,6,20,2016,1846-01-01,1870-06-20
48,FRN,220,France,1944,8,25,2016,12,31,2016,1944-08-25,2016-12-31


### `2)` Match with `societies` in database on `ccode`/`cow_code`
Dataframe uses `cow_code` as identifier, while the database uses an auto-generated `society_id`.  Match `cow_code` in `societies` dataframe with `ccode` in `states` dataframe so database inserts will use correct identifier.

##### Get societies from database, then merge on `cow_code`

In [75]:
query = db.run_query('SELECT society_id, common_name, cow_code FROM society')
societies = pd.DataFrame.from_dict(query['data'])
societies.sample(5)

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No connection could be made because the target machine actively refused it)

In [76]:
states = states.merge(societies, left_on='ccode', right_on='cow_code')

#### Test

In [77]:
# Make sure statenme and common_name roughly match
states.sample(10)

,stateabb,ccode,statenme,styear,stmonth,stday,endyear,endmonth,endday,version,start_date,end_date,society_id,common_name,cow_code
123,EQG,411,Equatorial Guinea,1968,10,12,2016,12,31,2016,1968-10-12,2016-12-31,67,Equatorial Guinea,411
67,POL,290,Poland,1918,11,3,1939,9,27,2016,1918-11-03,1939-09-27,174,Poland,290
193,QAT,694,Qatar,1971,9,3,2016,12,31,2016,1971-09-03,2016-12-31,177,Qatar,694
242,WSM,990,Samoa,1976,12,15,2016,12,31,2016,1976-12-15,2016-12-31,189,Samoa,990
160,SAF,560,South Africa,1920,1,10,2016,12,31,2016,1920-01-10,2016-12-31,203,South Africa,560
213,BNG,771,Bangladesh,1971,12,16,2016,12,31,2016,1971-12-16,2016-12-31,18,Bangladesh,771
224,SIN,830,Singapore,1965,8,9,2016,12,31,2016,1965-08-09,2016-12-31,197,Singapore,830
196,AFG,700,Afghanistan,1919,8,8,2016,12,31,2016,1919-08-08,2016-12-31,1,Afghanistan,700
21,HON,91,Honduras,1899,1,1,2016,12,31,2016,1899-01-01,2016-12-31,98,Honduras,91
50,LIE,223,Liechtenstein,1990,9,18,2016,12,31,2016,1990-09-18,2016-12-31,125,Liechtenstein,223


### `3)` Drop `stateabb`, `version`, `ccode`, `cow_code`, `statenme`, `common_name`, and date part columns.
Only `society_id`, `start_date`, and `end_date` are needed.

In [79]:
states.drop(columns=['stateabb','ccode','cow_code','statenme','common_name','version','styear','stmonth','stday','endyear','endmonth','endday'], inplace=True)

#### Test

In [80]:
states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_date  243 non-null    datetime64[ns]
 1   end_date    243 non-null    datetime64[ns]
 2   society_id  243 non-null    int64         
dtypes: datetime64[ns](2), int64(1)
memory usage: 7.6 KB


### `4)` Add column `society_group_id` and set to 6 (ID for 'COW State System') for all rows

In [81]:
states['society_group_id'] = 6

#### Test

In [88]:
print('Unique values:', states['society_group_id'].unique(), '\n')
states.info()

Unique values: [6] 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_date        243 non-null    datetime64[ns]
 1   end_date          243 non-null    datetime64[ns]
 2   society_id        243 non-null    int64         
 3   society_group_id  243 non-null    int64         
dtypes: datetime64[ns](2), int64(2)
memory usage: 9.5 KB


<a id='insert_cow'></a>
## Insert, COW State System Data into `society_to_society_group` table

In [ ]:
print('Expecting to insert:', states.shape[0], 'records')
insert = db.run_query(db.build_insert_query('society_to_society_group',states[['society_id','society_group_id','start_date','end_date']]))
print(insert['rows'], 'records inserted')

<a id='analyze_major'></a>
## Analysis, Major Powers Data

#### Issues Found:
`1)` 

<a id='clean_major'></a>
## Clean, Major Powers Data
### `1)` 

<a id='insert_major'></a>
## Insert, Major Powers Data into `society_to_society_group` table

## References

<ul>
    <li>Correlates of War Project. 2016. “State System Membership List, v2016.” Online,
        <a href='http://correlatesofwar.org' target='_new'>http://correlatesofwar.org</a>.</li>
    <li><a href='http://www.correlatesofwar.org/data-sets/downloadable-files/cow-country-codes' target='_new'>Correlates of War country codes</a></li>
</ul>